### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 25 ноября 2019, 15:00   
**Штраф за опоздание:** -2 балла после 15:00 25 ноября, -4 балла после 15:00 2 декабря, -6 баллов после 15:00 9 декабря  -8 баллов после 15:00 16 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0919, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на [wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) и [Speed Dating Data](https://cloud.mail.ru/public/8nHV/p6J7wY1y1)

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
#%%pycodestyle
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        
        #Критерии информативности 
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise
            
        # я выбираю:
        # l_c - число объектов в левом поддереве
        # r_c - число объектов в правом пддереве
        # l_s - массив количества объектов каждого класса в левом поддереве
        # r_s - массив количества объектов каждого класса в левом поддереве

        #F
    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return (1 - np.sum(l_s**2)/l_c - np.sum(r_s**2)/r_c) / (l_c+r_c) # Ваш код в 1 строчку
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return -(np.sum(l_s*np.log(l_s/l_c)) + np.sum(r_s*np.log(r_s/r_c))) / (l_c+r_c)# Ваш код в 1 строчку

    def __misclass(self, l_c, l_s, r_c, r_s):
        return (1 - (np.max(l_s)+np.max(r_s)) / (l_c+r_c) ) # Ваш код в 1 строчку

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))] # Ваш код в 1 строчку
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))] # Ваш код в 1 строчку

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature) # Ваш код
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]
    
    def __find_threshold(self, x, y):
        # Ваш код
        x_sorted, y_sorted = self.__sort_samples(x, y)
        # ищем индексы пограничных элементов
        index = np.where(y_sorted[:-1]!=y_sorted[1:])[0]+1
        if len(index) == 0:
            return np.inf, None
        errors = np.array([self.G_function(y_sorted[:i].size, np.bincount(y_sorted[:i]),
                                        y_sorted[i:].size, np.bincount(y_sorted[i:]))for i in index])
        i = np.argmin(errors)
        return errors[i], x_sorted[index[i]]

    def __fit_node(self, x, y, node_id, depth):
        # Ваш код
        classes_number = np.bincount(y)
        if ((self.max_depth is not None and depth == self.max_depth) or (x.shape[0] < self.min_samples_split)):
            self.tree[node_id] = self.LEAF_TYPE, np.argmax(classes_number), classes_number.astype(float)/classes_number.size
            return
        feature_ids = self.get_feature_ids(x.shape[1])
        errors = np.zeros(x.shape[1])
        thresholds = np.zeros(x.shape[1])
        for i in feature_ids:
            errors[i], thresholds[i] = self.__find_threshold(x[:, i], y)
        index = np.argmin(errors)
        if np.isnan(thresholds[index]):
            self.tree[node_id] = self.LEAF_TYPE, np.argmax(classes_number), classes_number.astype(float)/classes_number.size
            return
        x_left, x_right, y_left, y_right = self.__div_samples(x, y, index, thresholds[index])
        if ((y_left.shape[0] == 0) or (y_right.shape[0] == 0)):
            self.tree[node_id] = self.LEAF_TYPE, np.argmax(classes_number), classes_number.astype(float)/classes_number.size
            return
        self.tree[node_id] = self.NON_LEAF_TYPE, index, thresholds[index]
        self._importances_ (y, y_left, y_right, index)
        self.__fit_node(x_left, y_left, 2*node_id + 1, depth + 1)
        self.__fit_node(x_right, y_right, 2*node_id + 2, depth + 1)
        
    
    def fit(self, x, y):
        self.feature_importances_ = np.zeros(x.shape[1])
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 
        self.feature_importances_ /= np.linalg.norm(self.feature_importances_)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

    def _importances_ (self, y, y_left, y_right, index):
        y_frequence = np.unique(y, return_counts = True)
        y_left_frequence = np.unique(y_left, return_counts = True)
        y_right_frequence = np.unique(y_right, return_counts = True)
            
        if self.G_function == self.__gini:
            yy = 1 - np.sum(y_frequence[1]**2) / y.size**2
            yy_left = 1 - np.sum(y_left_frequence[1]**2) / y_left.size**2
            yy_right = 1 - np.sum(y_right_frequence[1]**2) / y_right.size**2

        elif self.G_function == self.__entropy:
            yy = - np.sum(y_frequence[1]/y.size * np.log(y_frequence[1]/y.size))
            yy_left = - np.sum(y_left_frequence[1]/y_left.size * np.log(y_left_frequence[1]/y_left.size))
            yy_right = - np.sum(y_right_frequence[1]/y_right.size * np.log(y_right_frequence[1]/y_right.size))

        elif self.G_function == self.__misclass:
            yy = 1 - np.max(y_frequence[1])/y.size
            yy_left = 1 - np.max(y_left_frequence[1])/y_l.size
            yy_right = 1 - np.max(y_right_frequence[1])/y_right.size
            
        self.feature_importances_[index] += (yy*y.size - yy_left*y_left.size - yy_right*y_right.size)

In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 12.7 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

CPU times: user 141 ms, sys: 15.6 ms, total: 156 ms
Wall time: 158 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8968253968253969

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8968253968253969

## Подготовка данных Speed Dating Data 

In [9]:
df = pd.read_csv('Speed Dating Data.csv', encoding='cp1251')
df = df.iloc[:, :97]
df = df.drop([
             'id', 
             'idg',
             'condtn',
             'round',
             'position',
             'positin1',
             'order',
             'partner',
             'age_o',
             'race_o', 
             'pf_o_att',
             'pf_o_sin', 
             'pf_o_int',
             'pf_o_fun',
             'pf_o_amb',
             'pf_o_sha',
             'dec_o',
             'attr_o',
             'sinc_o',
             'intel_o',
             'fun_o',
             'amb_o',
             'shar_o', 
             'like_o',
             'prob_o',
             'met_o',
             'sports',
             'tvsports',
             'exercise',
             'dining', 
             'museums',
             'art', 
             'hiking',
             'gaming', 
             'clubbing', 
             'reading', 
             'tv',
             'theater', 
             'movies', 
             'concerts', 
             'music',
             'shopping',
             'yoga',
             'career',
             'expnum'], axis=1)
df.drop_duplicates('iid').age.isnull().sum()
df = df.dropna(subset=['age'])

df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
df = df.drop(['field'], axis=1)
pd.get_dummies(df, columns=['field_cd'], prefix='field_cd', prefix_sep='=')
df = df.drop(['undergra'], axis=1)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)
df = df.dropna(subset=['imprelig', 'imprace'])
df = df.drop(['from', 'zipcode'], axis=1)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)


feat = ['iid', 'wave', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1',
        'amb1_1', 'shar1_1']

temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1',
          'shar1_1']].sum(axis=1)

df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1',
           'shar1_1']] = (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
                                     'fun1_1', 'amb1_1', 'shar1_1']].T /df.loc[:, 'temp_totalsum'].T).T * 100

feat = ['iid', 'wave', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1',
        'amb2_1', 'shar2_1']

temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                                        'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)

df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1',
           'shar2_1']] = (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                                     'fun2_1', 'amb2_1', 'shar2_1']].T / df.loc[:, 'temp_totalsum'].T).T * 100
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    df = df.drop(feat, axis=1)

df = df.drop(['wave'], axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']) \
            .drop(['gender'], axis=1).dropna()

df_female = df.query('gender == 0').drop_duplicates(subset=['iid']) \
            .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1).dropna()

df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)

X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=12)
df.head()

,iid,gender,pid,match,int_corr,samerace,age,field_cd,mn_sat,tuition,...,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,temp_totalsum
0,1,0,11.0,0,0.14,0,21.0,1.0,-999.0,-999.0,...,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,100.0
1,1,0,12.0,0,0.54,0,21.0,1.0,-999.0,-999.0,...,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,100.0
2,1,0,13.0,1,0.16,1,21.0,1.0,-999.0,-999.0,...,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,100.0
3,1,0,14.0,1,0.61,0,21.0,1.0,-999.0,-999.0,...,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,100.0
4,1,0,15.0,1,0.21,0,21.0,1.0,-999.0,-999.0,...,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,100.0


## Проверка скорости работы на Speed Dating Data 

In [10]:
# подсказки поменялись местами

In [11]:
# тут должен быть код типа f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

In [12]:
# тут должен быть код типа f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

In [13]:
%time clf.fit(X_train, y_train)

CPU times: user 109 ms, sys: 15.6 ms, total: 125 ms
Wall time: 128 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [14]:
%time my_clf.fit(X_train, y_train)

CPU times: user 20.1 s, sys: 31.2 ms, total: 20.1 s
Wall time: 20.4 s


## Проверка качества работы на Speed Dating Data

In [15]:
# тут должен быть код типа %time clf.fit(X_train, y_train)

In [16]:
# тут должен быть код типа %time my_clf.fit(X_train, y_train)

In [17]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.506745974176701

In [18]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5421644016701327

## Задание 3

## Задание 4

In [19]:
features_names = df_pair.columns[1:]
features = clf.feature_importances_
index = (-clf.feature_importances_).argsort()[0:10]
for i in index:
    print(features_names[i], ": ", features[i])

int_corr :  0.07334530082259362
fun1_1_f :  0.032048995184475626
amb3_1_f :  0.027630163376831204
sinc1_1 :  0.02715608346492082
age :  0.0263821001226739
imprelig_f :  0.025142948979455088
income_f :  0.025004522680492837
career_c_f :  0.023424610001320414
attr2_1_f :  0.023338135754337935
amb1_1_f :  0.022700209128178307


In [20]:
features_names = df_pair.columns[1:]
features = my_clf.feature_importances_
index = (-my_clf.feature_importances_).argsort()[0:10]
for i in index:
    print(features_names[i], ": ", features[i])

int_corr :  0.43865168191030246
imprelig_f :  0.2624457993150911
attr2_1_f :  0.2519442388434813
attr1_1 :  0.25068740815516866
attr3_1 :  0.22949214472238438
age :  0.2126720534968448
field_cd :  0.20650916799749047
field_cd_f :  0.20369114243890654
intel1_1_f :  0.20157877580771755
race :  0.1926961152970804


## Задание 5

In [21]:
params = {
    "min_samples_split": [i for i in range(2, 100)],
    "n_estimators": [i for i in range(1, 100)],
    "max_depth": [i for i in range(2, 100)],
}
model = RandomizedSearchCV(RandomForestClassifier(random_state=123), param_distributions=params, n_iter=100)
model.fit(X_train, y_train)

/home/linuxconfig/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs

In [22]:
print(model.best_params_)
print(f1_score(y_pred=model.predict(X_test), y_true=y_test, average='macro'))

{'n_estimators': 82, 'min_samples_split': 2, 'max_depth': 69}
0.5357455559854207
